In [28]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import json

In [29]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("API key loaded successfully")
else:
    print("Failed to load api key")

MODEL = "gpt-4.1-mini"
openai = OpenAI()

API key loaded successfully


In [30]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [31]:
def chat(messsage, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "system_message"}] + history + [{"role": "user", "content": messsage}]
    resposne = openai.chat.completions.create(model=MODEL, messages=messages)
    return resposne.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [32]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknonw ticket price")
    return f"The price of ticket to {destination_city} is {price}"

In [33]:
get_ticket_price("London")

Tool called for city London


'The price of ticket to London is $799'

In [34]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [35]:
tools = [{"type": "function", "function": price_function}]

In [36]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [37]:
# For single tool call. Cannot handle multiple tool call in a single request
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get("destination_city")
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }

    return response

In [38]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    resposne = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if resposne.choices[0].finish_reason == "tool_calls":
        message = resposne.choices[0].message
        resposne = handle_tool_call(message)
        messages.append(message)
        messages.append(resposne)

        resposne = openai.chat.completions.create(model=MODEL, messages=messages)

    return resposne.choices[0].message.content

In [39]:
# To handle multiple tool call in a single request but cannot handle multiple subsequent tool call
# Sample prompt - "Check the flight price for London and Paris"
def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get("destination_city")
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })

    return responses

In [40]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_call(message)
        messages.append(message)
        messages.extend(responses)

        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [41]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [42]:
# To handle multiple tool call in a single request or subsequent tool call request
# Sample prompt - "First check the price of flight to London and only if it's less than $1000 check the price for Paris"
def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get("destination_city")
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })

    return responses

In [43]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_call(message=message)
        messages.append(message)
        messages.extend(responses)

        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    return response.choices[0].message.content

In [44]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [45]:
import sqlite3

In [46]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)")
    conn.commit()

In [47]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting ticket price of {city}", flush=True)

    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT price FROM prices WHERE city = ?", (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is {result[0]}" if result else f"No price data available for {city}"

In [48]:
get_ticket_price("london")

DATABASE TOOL CALLED: Getting ticket price of london


'Ticket price to london is $799'

In [ ]:
def set_ticket_price(city, price):
    print(f"DATABASE TOOL CALLED: Setting price for {city}")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO prices(city, price) VALUES (?, ?)", (city.lower(), price,))
        conn.commit()

DATABASE TOOL CALLED: Setting price for Mumbai


In [50]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

for city, price in ticket_prices.items():
    set_ticket_price(city=city, price=price)

IntegrityError: UNIQUE constraint failed: prices.city

In [51]:
get_ticket_price("london")

DATABASE TOOL CALLED: Getting ticket price of london


'Ticket price to london is $799'

In [52]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [53]:
set_price_function = {
    "name": "set_ticket_price",
    "description": "Set the price for a destination city whose data is not available in the database",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The destination city whose price data is not available"
            },
            "price": {
                "type": "number",
                "description": "Return flight ticket price of the destination city"
            }
        },
        "required": ["destination_city", "price"],
        "additionaProperties": False
    }
}

In [54]:
tools.append({"type": "function", "function": set_price_function})

In [55]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'set_ticket_price',
   'description': 'Set the price for a destination city whose data is not available in the database',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The destination city whose price data is not available'},
     'price': {'type': 'number',
      'description': 'Return flight ticket price of the destination city'}},
    'required': ['destination_city', 'price'],
    'additionaProperties': False}}}]

In [60]:
def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "get_ticket_price":
            city = arguments.get("destination_city")
            price_details = get_ticket_price(city=city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })

        elif tool_call.function.name == "set_ticket_price":
            city = arguments.get("destination_city")
            price = arguments.get("price")
            set_ticket_price(city=city, price=price)
            responses.append({
                "role": "tool",
                "content": f"Ticket price of ${price} for {city} saved in database",
                "tool_call_id": tool_call.id
            })
        else:
            responses.append({
                "role": "tool",
                "content": "Apologies, tool not defined yet",
                "tool_call_id": tool_call.id
            })

    return responses

In [61]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Getting ticket price of Kolkata
DATABASE TOOL CALLED: Getting ticket price of Delhi
DATABASE TOOL CALLED: Getting ticket price of Delhi
